In [1]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
import sklearn


In [2]:
data = [] 
for line in open('data/review_train.json', 'r'): 
    data.append(json.loads(line))

In [3]:
dat=data[0:1000]

In [4]:
reviewslist = copy.deepcopy(dat)


texts=''
texts = reviewslist[ind]['text']
texts = texts.lower()
    
    #add NOT_
pattern = r'\.|\;|\!|\?|\,|\*|\)|\$|\(|\:|\'|\"|\[|\]|\%|\n'
list_text=re.split(pattern,texts)
list_text[2] 



for i in range(len(list_text)):
    print(i)
    list_text[i] = re.sub('\+','', list_text[i])
    list_text[i] = re.sub('\*','', list_text[i])
    list_text[i] = re.sub('\$','', list_text[i])
    list_text[i] = re.sub('\[','', list_text[i])
    list_text[i] = re.sub('\]','', list_text[i])
    list_text[i] = re.sub('\%','', list_text[i])
    list_text[i] = re.sub('\\\\','', list_text[i])
    matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
    if matchObj1 != None :
        sub=re.sub(r' ', " notadd", matchObj1.group(2))
        list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
        list_text[i] = re.sub(r'not ','', list_text[i],1)
        #list_text[i] = re.sub(',',' ', matchObj1.group(3))
        sent = sent + list_text[i] + ' '
    else:
        sent = sent + list_text[i] + ' '



In [ ]:
import re


REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(len(reviewslist)):
    texts=''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('were','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
#    texts = re.sub('won','will not', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        if matchObj1 != None :
            sub=re.sub(r' ', " notadd", matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        else:
            sent = sent + list_text[i] + ' '
   
    #print(sent)
    #print(texts)
    #print(texts)
    text_clean=preprocess_reviews(sent)
    #print(text_clean)
    reviewsTEXT_clean[ind]['text']=text_clean
    reviewsTEXT_clean[ind]['text_split']=text_clean.split()
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_split'])
    
#reviewsTEXT_clean[0]

In [305]:
#select words due to frequency order
AllFreq = reviewsTEXT_clean[0]['freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['freq']
    AllFreq.update(freqs)
    
wordList = sorted(AllFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
WORDS=[x[0] for x in wordList]
WORDS=WORDS[0:10000]


In [25]:
stop_words = set(stopwords.words('english')) 
negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn','not'])
stopWords=stop_words-negWords

WORDS=[x for x in WORDS if x not in stopWords]


In [26]:
WORDS

['good',
 'food',
 'place',
 'great',
 'time',
 'service',
 'one',
 'like',
 'back',
 'get',
 'would',
 'go',
 'no',
 'really',
 'us',
 'even',
 'also',
 'got',
 'well',
 'dont',
 'nice',
 'didnt',
 'best',
 'first',
 'staff',
 'notaddthe',
 'im',
 'ive',
 'always',
 'never',
 'chicken',
 'love',
 'could',
 'came']

In [27]:
#create list of texts
texts= [x['text'] for x in reviewsTEXT_clean]

In [28]:
#vectorizer = CountVectorizer(token_pattern=r'\b[^\d\W]+\b')
vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(texts)
transformer = TfidfTransformer()

tfidf = transformer.fit_transform(vectorizer.fit_transform(texts))

X=tfidf

<1000x10015 sparse matrix of type '<class 'numpy.int64'>'
	with 75521 stored elements in Compressed Sparse Row format>

In [29]:
X.df=pd.DataFrame(X.A,columns=vectorizer.get_feature_names())
Reviews.df=X.df[WORDS]
#WORDS
#X.df
Reviews.df

,good,food,place,great,time,service,one,like,back,get,...,staff,notaddthe,im,ive,always,never,chicken,love,could,came
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,1,2,1,0,2,...,0,0,2,1,1,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,7,1,0,0,0,2,...,0,1,0,0,1,2,0,0,4,0
5,3,2,1,0,0,1,0,1,0,0,...,0,0,1,0,0,0,2,0,0,0
6,0,0,1,0,1,0,2,4,0,1,...,0,0,1,0,1,0,1,0,1,2
7,0,2,1,0,0,1,0,0,1,0,...,1,1,0,1,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
#Add Business ID in the data frame
businessIDs=[x['business_id'] for x in reviewsTEXT_clean]
Reviews.df['business_id']=businessIDs
Reviews.df

In [ ]:
DataFrame.to_csv(Reviews.df,"reviews.csv",index=False)

In [30]:
a=[x['stars'] for x in reviewsTEXT_clean]

In [37]:
#random forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=123)
model.fit(df_imp, [x['stars'] for x in reviewsTEXT_clean])
pred = model.predict(df_imp)

In [38]:
np.sqrt(((a-pred)*(a-pred)).sum()/1000)

0.3924283374069717

In [39]:
#decision tree
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()   
dtc.fit(df_imp, [x['stars'] for x in reviewsTEXT_clean])
pred = dtc.predict(df_imp)

In [40]:
np.sqrt(((a-pred)*(a-pred)).sum()/1000)

0.382099463490856

In [41]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(df_imp, [x['stars'] for x in reviewsTEXT_clean])
pred = knn.predict(df_imp)

In [42]:
np.sqrt(((a-pred)*(a-pred)).sum()/1000)

1.6516658257650063